# Deploy to an batch endpoint

Imagine a health clinic takes patient measurements all day, saving the details for each patient in a separate file. Then overnight, the diabetes prediction model can be used to process all of the day's patient data as a batch, generating predictions that will be waiting the following morning so that the clinic can follow up with patients who are predicted to be at risk of diabetes. With Azure Machine Learning, you can accomplish this by creating a batch endpoint; and that's what you'll implement in this exercise.

## Before you start

You'll need the latest version of the  **azure-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.29.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /home/tshen/dp-100/mslearn-azure-ml/.venv/lib/python3.12/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [5]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

Config  

In [6]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
#Enter    details    of    your    Azure    Machine    Learning    workspace
subscription_id = 'c6cdfd9c-3767-4be7-a343-0acd27ddefb9'
resource_group = 'rg-dp100-l'
workspace = 'mlw-dp100-l'
#connect to the workspace
ml_client    =    MLClient(DefaultAzureCredential(),    subscription_id,
resource_group,    workspace)
ml_client

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f7d048c79b0>,
         subscription_id=c6cdfd9c-3767-4be7-a343-0acd27ddefb9,
         resource_group_name=rg-dp100-l,
         workspace_name=mlw-dp100-l)

## Register the model

Batch deployments can only deploy models registered in the workspace. You'll register an MLflow model, which is stored in the local `model` folder. 

In [7]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

model_name = 'diabetes-mlflow'
model = ml_client.models.create_or_update(
    Model(name=model_name, path='./model', type=AssetTypes.MLFLOW_MODEL)
)

Uploading model (0.0 MBs): 100%|██████████| 2106/2106 [00:00<00:00, 7149.07it/s]




## Create a batch endpoint

A batch endpoint is an HTTPS endpoint that applications can call to trigger a batch scoring job. A batch endpoint name needs to be unique within an Azure region. You'll use the `datetime` function to generate a unique name based on the current date and time. 

In [8]:
import datetime

endpoint_name = "batch-" + datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint_name

'batch-09201852074054'

To create an endpoint with the `BatchEndpoint` class, you need to specify the name and optionally a description. After creating an endpoint, you'll deploy a model to the endpoint.

In [9]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [10]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

/home/tshen/dp-100/mslearn-azure-ml/.venv/lib/python3.12/site-packages/azure/ai/ml/entities/_deployment/batch_deployment.py:137: UserWarning: This class is intended as a base class and it's direct usage is deprecated. Use one of the concrete implementations instead:
* ModelBatchDeployment - For model-based batch deployments
* PipelineComponentBatchDeployment - For pipeline component-based batch deployments
  warnings.warn(


<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [11]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [12]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

Uploading data (0.02 MBs): 100%|██████████| 17119/17119 [00:00<00:00, 58404.95it/s]




Data({'path': 'azureml://subscriptions/c6cdfd9c-3767-4be7-a343-0acd27ddefb9/resourcegroups/rg-dp100-l/workspaces/mlw-dp100-l/datastores/workspaceblobstore/paths/LocalUpload/f726f9a13c3cba9a7e260da3e6c74d8a391c555b50590a16c7f7726a4600ad69/data/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'patient-data-unlabeled', 'description': 'An unlabeled dataset for diabetes classification', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/c6cdfd9c-3767-4be7-a343-0acd27ddefb9/resourceGroups/rg-dp100-l/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-l/data/patient-data-unlabeled/versions/1', 'Resource__source_path': '', 'base_path': '/home/tshen/dp-100/mslearn-azure-ml/Labs/11', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f7ce0400500>, 'serialize': <msrest.serialization.Ser

In [13]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [14]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [15]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

Experiment,Name,Type,Status,Details Page
batch-09201852074054,batchjob-ab315047-d144-4a42-8fb5-97c7b0031065,pipeline,Preparing,Link to Azure Machine Learning studio


## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [17]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [18]:
import pandas as pd

score = pd.read_csv("predictions.csv", index_col=0, names=["patient", "prediction", "file"])
score

,prediction,file
patient,,
0,1,patient-data-0.csv
1,1,patient-data-0.csv
2,1,patient-data-0.csv
3,1,patient-data-0.csv
4,1,patient-data-0.csv
...,...,...
95,1,patient-data-3.csv
96,1,patient-data-3.csv
97,1,patient-data-3.csv
